In [77]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [78]:
transform =  transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [79]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [80]:
image, label = train_data[0]

In [81]:
image.size()
train_loader.dataset.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [82]:
class_names = ['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [84]:
class NeuralNet(nn.Module):
  
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3, 12, 5) # 5*5 as kernel moving along image and producing 12 feature maps as a result
    # (input size - kernel size) / stride (how much kernel moves by px) = 27 / 1 = 27 + 1 = 28 (shape)
    # 12 channels with 28 * 28 -> (12, 28, 28)
    self.pool = nn.MaxPool2d(2, 2) # takes 2*2 area and turns into 1px (4px to 1px) -> (12, 14, 14)
    self.conv2 = nn.Conv2d(12, 24, 5) # ... (24, 10, 10) -> (24, 5, 5) -> flatten (24 * 5 * 5)
    self.fc1 = nn.Linear(24 * 5 * 5, 120) # can use different numbers as long as shape compatible
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10) # but finish with 10 (classes)

  def forward(self, x): # applies ^ as input
    x = self.pool(F.relu(self.conv1(x))) # pool onto relu: 0 before 0 and x=y linear > 0
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [85]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [86]:
for epoch in range(30):
  print(f'Training epoch {epoch + 1}...')

  running_loss = 0.0

  for i, data in enumerate(train_loader):
    inputs, labels = data

    optimizer.zero_grad()

    outputs = net(inputs)

    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 1...
Loss: 2.1821
Training epoch 2...
Loss: 1.7315
Training epoch 3...
Loss: 1.5146
Training epoch 4...
Loss: 1.3931
Training epoch 5...
Loss: 1.3032
Training epoch 6...
Loss: 1.2227
Training epoch 7...
Loss: 1.1524
Training epoch 8...
Loss: 1.0913
Training epoch 9...
Loss: 1.0413
Training epoch 10...
Loss: 0.9963
Training epoch 11...
Loss: 0.9588
Training epoch 12...
Loss: 0.9220
Training epoch 13...
Loss: 0.8882
Training epoch 14...
Loss: 0.8561
Training epoch 15...
Loss: 0.8234
Training epoch 16...
Loss: 0.7936
Training epoch 17...
Loss: 0.7646
Training epoch 18...
Loss: 0.7378
Training epoch 19...
Loss: 0.7112
Training epoch 20...
Loss: 0.6844
Training epoch 21...
Loss: 0.6643
Training epoch 22...
Loss: 0.6408
Training epoch 23...
Loss: 0.6187
Training epoch 24...
Loss: 0.5992
Training epoch 25...
Loss: 0.5740
Training epoch 26...
Loss: 0.5536
Training epoch 27...
Loss: 0.5348
Training epoch 28...
Loss: 0.5117
Training epoch 29...
Loss: 0.4948
Training epoch 30...
Lo

In [87]:
torch.save(net.state_dict(), 'trained_net.pth')

# Load model

In [88]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

/var/folders/j8/rb8khncs0j73tq5y5x9g6d980000gn/T/ipykernel_2591/1690949604.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pt

<All keys matched successfully>

In [89]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
  for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

Accuracy: 68.11%


## Test with image not from dataset

In [92]:
new_transform = transforms.Compose([
  transforms.Resize((32, 32)),
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image = image.unsqueeze(0)
  return image

image_paths = ['./assets/640px-White-tailed_deer.jpg', './assets/frog-2_ver_1.jpg', './assets/airplane.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
  for image in images:
    output = net(image)
    _, predicted = torch.max(output, 1)
    print(f'Prediction: {class_names[predicted.item()]}')

Prediction: deer
Prediction: frog
Prediction: airplane
